In [ ]:
def get_multiO(m,N):
    omasks = config_RG_masks(m,N)
    mO=np.zeros((N,N))
    for i in range(len(omasks)):
        dim=len(omasks[i])
        Pt=np.zeros((N,dim))
        Pt[omasks[i]] = np.eye(dim)
        P=np.transpose(Pt)
        O=get_O(dim)
        mO+=np.dot(Pt,np.dot(O,P))
    return mO
mO=get_multiO(3,N_nod)
mOt=O=(torch.tensor(np.transpose(mO)).float()).to(device)

In [ ]:
def getproj(mask,N=N_nod):
    dim = len(mask)
    Pt=torch.zeros((N,dim))
    Pt[mask] = torch.eye(dim)
    return Pt

In [6]:
import numpy as np
def bininter(dim):
    P = np.zeros((2*dim,dim))
    mask = [2*i for i in range(dim)]
    mask1 = [2*i+1 for i in range(dim)]
    P[mask] = np.eye(dim)
    P[mask1] = 0.5 * np.eye(dim) + 0.5 * np.roll(np.eye(dim),1,axis=1)
    return P
print(bininter(4))    
    

[[1.  0.  0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  1. ]
 [0.5 0.  0.  0.5]]


In [ ]:
class KL_mix(nn.Module):
    
    def __init__(self,KL1,KL2,alpha=0.5):
        super().__init__()
        self.KL1 = KL1
        self.KL2 = KL2
        self.alpha = alpha
    
    def forward(self,x,log_abs_det):
        n_nod = x.shape[1]
        N2 = self.KL2.n_nod
        step = n_nod//N2
        x1 = x[:,::step]
        loss = (1-self.alpha) * self.KL1(x,log_abs_det) + self.alpha * self.KL2(x1,log_abs_det) 
        return loss

In [ ]:
class CubicCouplingLayer(nn.Module):
    def __init__(
        self,
        theta,
        split,
        swap: int,
        reg=1
    ):
        super().__init__()
        self.theta = theta
        self.split = split
        self.swap = swap
        self.reg=reg
        
    def g(self, z: torch.Tensor,params=torch.tensor([])) -> torch.Tensor:
        """g : z -> x. The inverse of f."""
        mask1=self.split[self.swap]
        mask2=self.split[(self.swap+1)%2]
        z1, z2 = z[:,mask1], z[:,mask2]
        z1=torch.cat((z1,params),dim=-1)
        t, s = self.theta(z1)
        x2 = z2 + torch.exp(s-self.reg) * (z2) ** 3 + t
        log_det = (torch.log(1+3 * torch.exp(s-self.reg) * (z2) ** 2)).sum(-1)
        z[:,mask2]=x2
        return z, log_det    



In [ ]:
class D(nn.Linear):
    def __init__(self,n):
        super().__init__(n,1,bias=False)
        self.n=n
        self.d_ind=[(n+1)*k for k in range(n)]
    
    def anti_flatten(self):
        n=self.n
        D=torch.zeros((n**2)).to(device)
        D[self.d_ind]=self.weight.to(device)
        D=torch.reshape(D,(n,n)).to(device)
        return D        
            
    def log_abs_det(self):
        ABS=torch.abs(self.weight)
        l=torch.log(ABS)
        lad=torch.sum(l)
        return lad
    
    def forward(self,x):
        D=(self.anti_flatten()).to(x.device)
        return torch.matmul(x,D)
    
    def g(self,z,params):
        lad=self.log_abs_det()
        return self.forward(z),  lad * torch.ones((z.shape[0])).to(z.device)
        
    def adj(self,mat):
        Lw=self.anti_flatten()
        Lwt=torch.t(Lw)
        D=torch.matmul(Lwt,torch.matmul(mat,Lw))
        return D
    